In [3]:
import simplejson as json

def get_tweet(doc):
    tweetItem = {}

#Available tweet data
    tweetItem['text'] = doc['text']

    #hashtags are identified and provided as a field in the tweet
    tweetItem['hashtags'] = map(lambda x: x['text'],doc['entities']['hashtags'])

    # user_mentiones are identified and provided as a field 
    tweetItem['user_mentions'] = map(lambda x: x['screen_name'],doc['entities']
    ['user_mentions'])
    # symbols e.g. $APPL are identified and provided as a field 
    tweetItem['symbols'] = map(lambda x: x['text'],doc['entities']['symbols'])
    tweetItem['coordinates'] = doc['coordinates']
    tweetItem['user_id'] = doc['user']['id']
    tweetItem['user_name'] = doc['user']['name']
    try:
        tweetItem['retweet_id'] = doc['retweeted_status']['id']

    except KeyError as e:
        tweetItem['retweet_id'] = 0
        pass

    return tweetItem

ModuleNotFoundError: No module named 'simplejson'

In [2]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/usr/share/stanford-ner/stanford-ner.jar',encoding='utf-8')
#a tweet by Donald Trump
text = 'Just had a very good call with @SwedishPM Stefan Löfven who assured me that American citizen A$AP Rocky will be treated fairly. Likewise, I assured him that A$AP was not a flight risk and offered to personally vouch for his bail, or an alternative....'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

LookupError: Could not find stanford-ner.jar jar file at /usr/share/stanford-ner/stanford-ner.jar

In [ ]:
def cluster_text(dist_text,processedTweets,dist_threshold_txt):
    cluster_id = 1
    for i in range(dist_text.shape[0]):
        try:
            doc_dist_array_txt = dist_text[i,]
            # identify document ids where the tweets are similar
            similarDocs = np.where(doc_dist_array_txt <= dist_threshold_txt)[0]
            processedTweets.ix[processedTweets.index.isin(similarDocs) & processedTweets['cluster_ref'].isin([None]), 'cluster_ref'] = cluster_id
            cluster_id = cluster_id + 1
        except ValueError:
            continue

    return processedTweets



# %%%%%%%%% Calculate Cosine distance of documents %%%%%%%%%%%%%%

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def get_clusters(processedTweets, similarity_threshold):
    tweets = processedTweets['text'].tolist()

    #Define count vectorizer parameters
    vectorizer =  CountVectorizer(max_df=1.0, min_df=1, stop_words=stopwords, tokenizer=tokenize_only) 

    # Get document term frequency matix
    dtf_matrix = vectorizer.fit_transform(tweets) #fit the vectorizer to tweets which does not contain'htttp' and 'RT'
    dist_text = np.around(abs(1 - cosine_similarity(dtf_matrix)),2)

    # --------1D clustering based on distance measure
    # Pre clustering setup
    processedTweets['cluster_ref'] = None # Tweets that are 1-similarithy% similar, as defined by dist_threshold
    # Clustering of tweets based on text
    processedTweets = cluster_text(dist_text,processedTweets,dist_threshold_txt = (1-similarity_threshold))

    return processedTweets